In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
df = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/processed/cleaned_data.csv")
df.head()

,Total_Accidents,Total_Release_BBLS,Total_Recovered_BBLS,Avg_Prop_Damage,Avg_Env_Damage,Total_Fatalities,Total_Injuries,Operational_Rigs,Distillation_Capacity,Refinery_Inputs,%_Utilization_of_Refinery_Operable_Capacity,Date
0,34,7562.18,2590.44,16872.529412,236234.205882,0,0,433.0,17597.0,14065.0,79.9,2010-01-01
1,16,13791.04,29.95,78075.062500,15665.500000,0,0,446.0,17584.0,14267.0,81.1,2010-02-01
2,32,3663.14,2689.68,79171.843750,30998.281250,0,0,471.0,17584.0,14630.0,83.2,2010-03-01
3,32,3054.18,392.01,104276.687500,148810.875000,0,1,508.0,17589.0,15592.0,88.7,2010-04-01
4,30,5102.98,2841.72,949891.000000,75947.833333,0,0,541.0,17589.0,15510.0,88.2,2010-05-01


## Creating the Strain Index Column

### Strain_Index = 1 - Refinery_Inputs/Distillation Capacity

In [3]:
#Creating the target Variable (Strain_Index)

df["Strain_Index"] = 1 - (df["Refinery_Inputs"] / df["Distillation_Capacity"])


In [4]:
#Creating Lagged Features for previous month, 3 months ago, 6 months ago and 12 months ago

engineered_df = df.copy(deep=True)

for col in ["Strain_Index", "Total_Accidents", "Operational_Rigs"]:
    for lag in [1,3,6,12]:
        engineered_df[f"{col}_lag{lag}"] = engineered_df[col].shift(lag)

In [5]:
#Creating rolling window features for 3, 6 and 12 months

for col in ["Strain_Index", "Total_Accidents", "Total_Release_BBLS", "Operational_Rigs"]:
    for w in [3,6,12]:
        engineered_df[f"{col}_roll{w}_mean"] = engineered_df[col].shift(1).rolling(w).mean()
        engineered_df[f"{col}_roll{w}_sum"] = engineered_df[col].shift(1).rolling(w).sum()


In [6]:
#Creating the new target column (Strain_Index + 1) meaning next month strain_index

engineered_df["target_strain_index"] = engineered_df["Strain_Index"].shift(-1)


In [7]:
engineered_df.head(10)

,Total_Accidents,Total_Release_BBLS,Total_Recovered_BBLS,Avg_Prop_Damage,Avg_Env_Damage,Total_Fatalities,Total_Injuries,Operational_Rigs,Distillation_Capacity,Refinery_Inputs,%_Utilization_of_Refinery_Operable_Capacity,Date,Strain_Index,Strain_Index_lag1,Strain_Index_lag3,Strain_Index_lag6,Strain_Index_lag12,Total_Accidents_lag1,Total_Accidents_lag3,Total_Accidents_lag6,Total_Accidents_lag12,Operational_Rigs_lag1,Operational_Rigs_lag3,Operational_Rigs_lag6,Operational_Rigs_lag12,Strain_Index_roll3_mean,Strain_Index_roll3_sum,Strain_Index_roll6_mean,Strain_Index_roll6_sum,Strain_Index_roll12_mean,Strain_Index_roll12_sum,Total_Accidents_roll3_mean,Total_Accidents_roll3_sum,Total_Accidents_roll6_mean,Total_Accidents_roll6_sum,Total_Accidents_roll12_mean,Total_Accidents_roll12_sum,Total_Release_BBLS_roll3_mean,Total_Release_BBLS_roll3_sum,Total_Release_BBLS_roll6_mean,Total_Release_BBLS_roll6_sum,Total_Release_BBLS_roll12_mean,Total_Release_BBLS_roll12_sum,Operational_Rigs_roll3_mean,Operational_Rigs_roll3_sum,Operational_Rigs_roll6_mean,Operational_Rigs_roll6_sum,Operational_Rigs_roll12_mean,Operational_Rigs_roll12_sum,target_strain_index
0,34,7562.18,2590.44,16872.529412,2.362342e+05,0,0,433.0,17597.0,14065.0,79.9,2010-01-01,0.200716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.188637
1,16,13791.04,29.95,78075.062500,1.566550e+04,0,0,446.0,17584.0,14267.0,81.1,2010-02-01,0.188637,0.200716,NaN,NaN,NaN,34.0,NaN,NaN,NaN,433.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.167994
2,32,3663.14,2689.68,79171.843750,3.099828e+04,0,0,471.0,17584.0,14630.0,83.2,2010-03-01,0.167994,0.188637,NaN,NaN,NaN,16.0,NaN,NaN,NaN,446.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.113537
3,32,3054.18,392.01,104276.687500,1.488109e+05,0,1,508.0,17589.0,15592.0,88.7,2010-04-01,0.113537,0.167994,0.200716,NaN,NaN,32.0,34.0,NaN,NaN,471.0,433.0,NaN,NaN,0.185782,0.557347,NaN,NaN,NaN,NaN,27.333333,82.0,NaN,NaN,NaN,NaN,8338.786667,25016.36,NaN,NaN,NaN,NaN,450.000000,1350.0,NaN,NaN,NaN,NaN,0.118199
4,30,5102.98,2841.72,949891.000000,7.594783e+04,0,0,541.0,17589.0,15510.0,88.2,2010-05-01,0.118199,0.113537,0.188637,NaN,NaN,32.0,16.0,NaN,NaN,508.0,446.0,NaN,NaN,0.156723,0.470168,NaN,NaN,NaN,NaN,26.666667,80.0,NaN,NaN,NaN,NaN,6836.120000,20508.36,NaN,NaN,NaN,NaN,475.000000,1425.0,NaN,NaN,NaN,NaN,0.093021
5,29,6210.06,1475.72,25189.896552,4.999562e+05,0,0,566.0,17523.0,15893.0,90.7,2010-06-01,0.093021,0.118199,0.167994,NaN,NaN,30.0,32.0,NaN,NaN,541.0,471.0,NaN,NaN,0.133243,0.399729,NaN,NaN,NaN,NaN,31.333333,94.0,NaN,NaN,NaN,NaN,3940.100000,11820.30,NaN,NaN,NaN,NaN,506.666667,1520.0,NaN,NaN,NaN,NaN,0.088326
6,29,23690.21,18651.67,133785.241379,2.193182e+07,1,1,591.0,17594.0,16040.0,91.2,2010-07-01,0.088326,0.093021,0.113537,0.200716,NaN,29.0,32.0,34.0,NaN,566.0,508.0,433.0,NaN,0.108252,0.324756,0.147017,0.882103,NaN,NaN,30.333333,91.0,28.833333,173.0,NaN,NaN,4789.073333,14367.22,6563.930000,39383.58,NaN,NaN,538.333333,1615.0,494.166667,2965.0,NaN,NaN,0.108730
7,27,4674.29,1148.40,265007.925926,1.936178e+05,0,0,644.0,17594.0,15681.0,89.1,2010-08-01,0.108730,0.088326,0.118199,0.188637,NaN,29.0,30.0,16.0,NaN,591.0,541.0,446.0,NaN,0.099848,0.299545,0.128285,0.769713,NaN,NaN,29.333333,88.0,28.000000,168.0,NaN,NaN,11667.750000,35003.25,9251.935000,55511.61,NaN,NaN,566.000000,1698.0,520.500000,3123.0,NaN,NaN,0.135330
8,29,8584.16,7893.32,112700.793103,5.840972e+05,0,0,668.0,17594.0,15213.0,86.5,2010-09-01,0.135330,0.108730,0.093021,0.167994,NaN,27.0,29.0,32.0,NaN,644.0,566.0,471.0,NaN,0.096692,0.290076,0.114968,0.689806,NaN,NaN,28.333333,85.0,29.833333,179.0,NaN,NaN,11524.853333,34574.56,7732.476667,46394.86,NaN,NaN,600.333333,1801.0,553.500000,3321.0,NaN,NaN,0.174692
9,10,10619.68,10035.48,5110.000000,3.497990e+04,0,0,693.0,17528.0,14466.0,82.5,20

In [8]:
#Dropping all NaNs

engineered_df = engineered_df.dropna().reset_index(drop=True)

In [9]:
#Creating our train, Evaluation and Holdout datasets

engineered_df = engineered_df.sort_values("Date").reset_index(drop=True)

#2010-2020
train_df = engineered_df[engineered_df["Date"] < "2021-01-01"]

#2020-2023
eval_df = engineered_df[(engineered_df["Date"] >= "2021-01-01") & (engineered_df["Date"] < "2024-01-01")]

#2024-2025
holdout_df = engineered_df[engineered_df["Date"] >= "2024-01-01"]



In [10]:
#Saving final datasets
import os

data_path = os.path.join("..", "data", "final_datasets")
os.makedirs(data_path, exist_ok=True)

datasets = {
    "train_df": train_df,
    "eval_df": eval_df,
    "holdout_df": holdout_df
}

for name, df in datasets.items():
    save_path = os.path.join(data_path, f"{name}.csv")
    df.to_csv(save_path, index=False)
    print(f"Saved {name}.csv to {save_path}")
print("All datasets saved")

Saved train_df.csv to ../data/final_datasets/train_df.csv
Saved eval_df.csv to ../data/final_datasets/eval_df.csv
Saved holdout_df.csv to ../data/final_datasets/holdout_df.csv
All datasets saved
